In [1]:
import sys
print(sys.version)

import sklearn.ensemble
import shap
import numpy as np
import numba
import time
import json
import xgboost
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import shap.benchmark

from shap import TreeExplainer as TreeCext

from shap.explainers.pytree import *

import lightgbm as lgb


3.10.6 (main, Nov 14 2022, 16:10:14) [GCC 11.3.0]


/home/jarek/projects/shap/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
np.set_printoptions(threshold=sys.maxsize)
# X,y = shap.datasets.communitiesandcrime()
X,y = shap.datasets.boston()
print(X.columns.values.tolist())
print(len(X.columns.values.tolist()))

print(X.shape)
print(y.shape)

['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT']
13
(506, 13)
(506,)


In [4]:
def run_explain(name, data, explainer, vals=False):
    measure_time = time.perf_counter
    start = measure_time()
    values = explainer(data)
    end = measure_time()
    print("name: {}, time: {}".format(name, end - start))
    return [name, values if vals else [], end - start]

def run_diff_explainers(model, data):

#     # use an independent masker
#     masker = shap.maskers.Independent(X_train)
#     pmasker = shap.maskers.Partition(X_train)
    explainers = [
        ("Tree", shap.explainers.Tree(model)),
        ("TreeBanz", shap.explainers.Tree(model,  use_banz=True)),
        ("NEW TreeBanz", shap.explainers.Tree(model,  use_banz=True, change_deltas=True)),
    #     ("Tree approx.", shap.explainers.Tree(model, masker, approximate=True)),
    #     ("Exact", shap.explainers.Exact(model.predict, masker)),
    ]

    return [run_explain(name, data, exp) for name, exp in explainers]

In [5]:
def run_explain2(name, data, explainer):
    measure_time = time.perf_counter
    start = measure_time()
    values = explainer(data)
    end = measure_time()
    print("name: {}, time: {}".format(name, end - start))
    return [name, values, end - start]

def run_diff_explainers2(model, data):

#     # use an independent masker
#     masker = shap.maskers.Independent(X_train)
#     pmasker = shap.maskers.Partition(X_train)
    explainers = [
        ("Tree", shap.explainers.Tree(model)),
        ("TreeBanz", shap.explainers.Tree(model,  use_banz=True)),
        ("NEW TreeBanz", shap.explainers.Tree(model,  use_banz=True, change_deltas=True)),
    #     ("Tree approx.", shap.explainers.Tree(model, masker, approximate=True)),
    #     ("Exact", shap.explainers.Exact(model.predict, masker)),
    ]

    return [run_explain2(name, data, exp) for name, exp in explainers]

In [6]:
# X,y = shap.datasets.boston()
# X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000, max_depth=13)
# model.fit(X_train, y_train)

# TreeBanzCext = shap.explainers.Tree(model, use_banz=True, change_deltas=True)
# TreeBanzCext(X_test)

1
# ex = TreeExplainer(model)
# res_banz_py = ex.banz_values(X_test)


1

In [7]:
# Zobaczmy na tym datasecie jaka glebokosc daje najlepsze wyszkolenie

# X,y = shap.datasets.boston()
# X = X.values
# X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# scores = {}
# mean = 5

# for i in range(1,13):
#     pass
#     print(i)
#     scores[i] = 0
#     for j in range(mean):
#         model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000, max_depth=i)
#         model.fit(X_train, y_train)
#         scores[i] = scores[i] + model.score(X_test, y_test)
#     scores[i] = scores[i] / mean

# scores

In [8]:
def item_generator(json_input, lookup_key):
    if isinstance(json_input, dict):
        for k, v in json_input.items():
            if k == lookup_key:
                yield v
            else:
                yield from item_generator(v, lookup_key)
    elif isinstance(json_input, list):
        for item in json_input:
            yield from item_generator(item, lookup_key)

def tree_depth(json_text):
    json_input = json.loads(json_text)
    return max(list(item_generator(json_input, 'depth'))) + 1


In [9]:
import shap.datasets as ds

def a(x):
    X, y = x
    print(X.shape)

a(ds.adult())
a(ds.boston())
a(ds.communitiesandcrime())
a(ds.corrgroups60())
a(ds.diabetes())
a(ds.imagenet50())
# a(ds.imdb())
a(ds.independentlinear60())
a(ds.iris())
a(ds.nhanesi())

(32561, 12)
(506, 13)
(1994, 101)
(1000, 60)
(442, 10)
(50, 224, 224, 3)
(1000, 60)
(150, 4)
(14264, 79)


In [10]:
# Zobaczmy na tym datasecie jaka glebokosc daje najlepsze wyszkolenie

X,y = shap.datasets.nhanesi()
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7, test_size=0.2)

xgb_full = xgboost.DMatrix(X, label=y)
xgb_train = xgboost.DMatrix(X_train, label=y_train)
xgb_test = xgboost.DMatrix(X_test, label=y_test)

scores = {}
mean_depths = {}
stddev_depths = {}
full_depths = {}

mean = 10

for i in range(1,2,1):
# for i in range(1,32,1):
    print(i)
    scores[i] = 0
    mean_depths[i] = 0
    stddev_depths[i] = 0
    for j in range(mean):
        params = {
            "eta": 0.002,
            "max_depth": 3,
            "objective": "survival:cox",
            "subsample": 0.5
        }
#         model = xgboost.train(params, xgb_train, 10000, evals = [(xgb_test, "test")], verbose_eval=1000)
        model = xgboost.XGBRegressor(n_estimators=1000, max_depth=i, subsample=0.5)
#         X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, early_stopping_rounds=10)
        scores[i] = scores[i] + model.score(X_test, y_test)
        
        booster = model.get_booster()
        tree_df = booster.trees_to_dataframe()
#         print("rysuje drzewo")
#         xgboost.plotting.plot_tree(booster)
#         print(booster.get_dump(dump_format = "json").head())
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        full_depths[i] = depths
        mean_depths[i] = mean_depths[i] + np.mean(depths)
        stddev_depths[i] = stddev_depths[i] + np.std(depths)
        
    scores[i] = scores[i] / mean
    mean_depths[i] = mean_depths[i] / mean
    stddev_depths[i] = stddev_depths[i] / mean

print(scores)
# print(mean_depths)
# print(stddev_depths)


1
{1: 0.35595893770551873}


In [11]:
#przyklad lundberga:
X,y = shap.datasets.nhanesi()
X_display,y_display = shap.datasets.nhanesi(display=True) # human readable feature values

xgb_full = xgboost.DMatrix(X, label=y)

# create a train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)
xgb_train = xgboost.DMatrix(X_train, label=y_train)
xgb_test = xgboost.DMatrix(X_test, label=y_test)

# use validation set to choose # of trees
params = {
    "eta": 0.002,
    "max_depth": 3,
    "objective": "survival:cox",
    "subsample": 0.5
}
# model_train = xgboost.train(params, xgb_train, 10000, evals = [(xgb_test, "test")], verbose_eval=1000)

# train final model on the full data set
params = {
    "eta": 0.002,
    "max_depth": 3, 
    "objective": "survival:cox",
    "subsample": 0.5
}
model_full = xgboost.train(params, xgb_full, 5000, evals = [(xgb_full, "test")], verbose_eval=1000)

def c_statistic_harrell(pred, labels):
    total = 0
    matches = 0
    for i in range(len(labels)):
        for j in range(len(labels)):
            if labels[j] > 0 and abs(labels[i]) > labels[j]:
                total += 1
                if pred[j] > pred[i]:
                    matches += 1
    return matches/total

# see how well we can order people by survival
c_statistic_harrell(model_full.predict(xgb_test, ntree_limit=5000), y_test)

[0]	test-cox-nloglik:9.28408
[1000]	test-cox-nloglik:8.60854
[2000]	test-cox-nloglik:8.53116
[3000]	test-cox-nloglik:8.49403
[4000]	test-cox-nloglik:8.47058
[4999]	test-cox-nloglik:8.45255


0.8354598601693753

In [12]:
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import r2_score

res = model_full.predict(xgb_test)

print(np.sqrt(MSE(res, y_test)))
print(r2_score(res, y_test))

16.95208156660044
-92.30155036647064


In [13]:
import pandas as pd

covid = pd.read_csv('../../data/Covid Data.csv')

print(covid.shape)

cols = ['PNEUMONIA','DIABETES', 'COPD', 'ASTHMA', 'INMSUPR','HIPERTENSION', 
        'OTHER_DISEASE', 'CARDIOVASCULAR', 'OBESITY','RENAL_CHRONIC', 'TOBACCO']
for col in cols :
    covid = covid[(covid[col] == 1)|(covid[col] == 2)]
    
covid['DEATH'] = [2 if row=='9999-99-99' else 1 for row in covid['DATE_DIED']]

covid['DEATH'].value_counts()

covid.drop(columns=['INTUBED','ICU','DATE_DIED'],inplace=True)

covid['PREGNANT'] = covid['PREGNANT'].replace(97,2)
covid['PREGNANT'] = covid['PREGNANT'].replace(98,2)

y = covid['CLASIFFICATION_FINAL']
X = covid.drop('CLASIFFICATION_FINAL', axis=1)

y.head()

(1048575, 21)


0    3
1    5
2    3
3    7
4    3
Name: CLASIFFICATION_FINAL, dtype: int64

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
print("training")
log_reg.fit(X_train, y_train)
print("testing")
print("Logistic Regression Accuracy :", log_reg.score(X_test, y_test))

training
testing
Logistic Regression Accuracy : 0.5364896040111008


In [15]:
i=13
model = sklearn.ensemble.RandomForestClassifier(max_depth=i)
model.fit(X_train, y_train)
print("{}: {}\n--------------------------------".format(i, model.score(X_test, y_test)))
model_default = sklearn.ensemble.RandomForestClassifier()
model_default.fit(X_train, y_train)
model_default.score(X_test, y_test)

13: 0.5461856987479942
--------------------------------


0.5266959630494901

In [16]:
X_test.shape

(205031, 18)

In [17]:
for i in range(25,26,20):
    X_2 = X_test[:i]
    print("{}: {}".format(i, X_2.shape))
    start = time.time()
    run_diff_explainers(model, X_2)
    run_diff_explainers(model_default, X_2)
    print(time.time()-start)
    print("------------------------")

25: (25, 18)
ile drzew cext? 100
ile drzew cext? 100
ile drzew cext? 100
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
name: Tree, time: 3.8095137199998135
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: TreeBanz, time: 1.1749337759993068
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: NEW TreeBanz, time: 1.1880112610015203
ile drzew cext? 100
ile drzew cext? 100
ile drzew cext? 100
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
name: Tree, time: 137.19052676699903
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: TreeBanz, time: 14.832690563998767
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: NEW TreeBanz, time: 15.107441946000108
178.2694239616394
------------------------


In [18]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

# model = xgboost.XGBClassifier(max_depth=13)
model_xgb = xgboost.XGBClassifier()
start = time.time()
model_xgb.fit(X_train, y_train)
end = time.time() - start
print("time to train: {}".format(end))
model_xgb.score(X_test, y_test)

time to train: 71.26990294456482


0.5474489223580824

In [19]:
X_test.shape

(205031, 18)

In [20]:
run_diff_explainers(model_xgb, X_test)
13

ile drzew cext? 700
ile drzew cext? 700
ile drzew cext? 700
uruchamiam z BANZ=False
name: Tree, time: 106.75238401099887
uruchamiam z BANZ=True
name: TreeBanz, time: 107.43274779199965
uruchamiam z BANZ=True
name: NEW TreeBanz, time: 109.83415282900023


13

In [21]:
from sklearn.datasets import load_wine
X, y = load_wine(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

model = sklearn.ensemble.RandomForestClassifier()
start = time.time()
model.fit(X_train, y_train)
end = time.time() - start
print("time to train: {}".format(end))
model.score(X_test, y_test)

time to train: 0.06077218055725098


0.9814814814814815

In [22]:
run_diff_explainers(model, X)
13

ile drzew cext? 100
ile drzew cext? 100
ile drzew cext? 100
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
name: Tree, time: 0.010936492999462644
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: TreeBanz, time: 0.01636628199958068
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: NEW TreeBanz, time: 0.01647013399997377


13

In [23]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler

df = pd.read_csv('../../data/Base.csv')
# remove "device_fraud_count"
df = df.drop(['device_fraud_count'], axis=1, errors='ignore') # It's 0 for all rows

df['fraud_bool'].value_counts()

X = df.drop(['fraud_bool'], axis=1)
y = df['fraud_bool']


# Train test split by 'month', month 0-5 are train, 6-7 are test data as proposed in the paper
X_train = X[X['month']<6]
X_test = X[X['month']>=6]
y_train = y[X['month']<6]
y_test = y[X['month']>=6]

X_train.drop('month', axis=1, inplace=True)
X_test.drop('month', axis=1, inplace=True)

# y_train = le.fit_transform(y_train)
# y_test = le.fit_transform(y_test)

# onehot encoding for categorical variables
s = (X_train.dtypes == 'object')
object_cols = list(s[s].index)

ohe = OneHotEncoder(sparse=False, handle_unknown='ignore')
ohe_cols_train = pd.DataFrame(ohe.fit_transform(X_train[object_cols]))
ohe_cols_test = pd.DataFrame(ohe.transform(X_test[object_cols]))

ohe_cols_train.index = X_train.index
ohe_cols_test.index = X_test.index

num_X_train = X_train.drop(object_cols, axis=1)
num_X_test = X_test.drop(object_cols, axis=1)

X_train = pd.concat([num_X_train, ohe_cols_train], axis=1)
X_test = pd.concat([num_X_test, ohe_cols_test], axis=1)

# Scale data
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)



In [24]:
model = sklearn.ensemble.RandomForestClassifier(max_depth=13)
start = time.time()
model.fit(X_train, y_train)
end = time.time() - start
print("time to train: {}".format(end))
model.score(X_test, y_test)

time to train: 121.02144479751587


0.9859617288828404

In [25]:
print(X_test.shape)
run_diff_explainers(model, X_test[:100])
13

(205011, 50)
ile drzew cext? 100
ile drzew cext? 100
ile drzew cext? 100
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
name: Tree, time: 6.904505259999496
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: TreeBanz, time: 2.6691520599997602
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: NEW TreeBanz, time: 2.6810266280008364


13

In [26]:
depths = [x.get_depth() for x in model.estimators_]
print(np.mean(depths))
print(np.std(depths))

13.0
0.0


In [27]:
model = sklearn.ensemble.RandomForestClassifier(max_depth=13)
start = time.time()
model.fit(X_train, y_train)
end = time.time() - start
print("time to train: {}".format(end))
print("score:{}".format(model.score(X_test, y_test)))

time to train: 146.68317079544067
score:0.9859617288828404


In [28]:
print(X_test.shape)
run_diff_explainers(model, X_test[:1000])

depths = [x.get_depth() for x in model.estimators_]
print(np.mean(depths))
print(np.std(depths))

(205011, 50)
ile drzew cext? 100
ile drzew cext? 100
ile drzew cext? 100
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
name: Tree, time: 73.15871214700019
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: TreeBanz, time: 27.942151048000596
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: NEW TreeBanz, time: 29.297103109998716
13.0
0.0


In [29]:
run_diff_explainers(model, X_test[:10])
13

ile drzew cext? 100
ile drzew cext? 100
ile drzew cext? 100
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
name: Tree, time: 0.7141172869996808
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: TreeBanz, time: 0.26969660100076
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
name: NEW TreeBanz, time: 0.2742919160009478


13

In [30]:
X_train.values

AttributeError: 'numpy.ndarray' object has no attribute 'values'

In [ ]:
model = lgb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)],
          verbose=20,eval_metric='logloss')
print('Testing accuracy {:.4f}'.format(model.score(X_test,y_test)))

In [ ]:
run_diff_explainers(model, X_test)

In [ ]:
model_xgb = xgboost.XGBClassifier()
start = time.time()
model_xgb.fit(X_train, y_train)
end = time.time() - start
print("time to train: {}".format(end))
model_xgb.score(X_test, y_test)

In [ ]:
run_diff_explainers(model_xgb, X_test)


In [ ]:
import catboost
model_cat = catboost.CatBoostClassifier()
start = time.time()
model_cat.fit(X_train, y_train)
end = time.time() - start
print("time to train: {}".format(end))
model_cat.score(X_test, y_test)

In [ ]:
run_diff_explainers(model_cat, X_train)

In [ ]:
# koniec czesci nowej

In [ ]:
# narysujmy sobie zaleznosc wytrenowania i glebokosci
sns.set()
n = max(scores) + 1
# n = len(scores) + 1
to_plot_scores = [0] * n
to_plot_mean = [0] * n
to_plot_stddev = [0] * n

for i in scores.keys():
    print(i)
    to_plot_scores[i] = scores[i]
    to_plot_mean[i] = mean_depths[i]
    to_plot_stddev[i] = stddev_depths[i]
    
to_plot_scores.remove(0)
to_plot_mean.remove(0)
to_plot_stddev.remove(0)

plt.title("Average score of models")
plt.ylabel("Accuracy in %")
plt.xlabel("Max tree depth")
plt.plot(to_plot_scores)
plt.show()
    
# ax = sns.heatmap(quality)
# plt.show()
# print(to_plot_mean)
# print(to_plot_stddev)

plt.title("Mean depth of tree in ensemble")
plt.ylabel("Mean depth")
plt.xlabel("Max tree depth")
plt.errorbar(range(len(to_plot_mean)), to_plot_mean, to_plot_stddev, linestyle='None', marker='^')
plt.show()

In [ ]:
# sprobujmy odtworzyc ten rezultat

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)

model = xgboost.XGBRegressor(n_estimators=5000, max_depth=2, subsample=0.5)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, early_stopping_rounds=10)
res = model.predict(X_test)

c_statistic_harrell(model.predict(X_test), y_test)

In [ ]:
for i in range(1,10):
    model = xgboost.XGBRegressor(n_estimators=1000, max_depth=i, subsample=0.5)
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=False, early_stopping_rounds=10)
    res = model.predict(X_test)

    print("{}: {}".format(i, c_statistic_harrell(model.predict(X_test), y_test)))

In [ ]:
scores = {}
mean_depths = {}
stddev_depths = {}
full_depths = {}

mean = 10

for i in range(1,32):
    print(i)
    scores[i] = 0
    mean_depths[i] = 0
    stddev_depths[i] = 0
    for j in range(mean):
        model = xgboost.XGBRegressor(n_estimators=(100+i*100), max_depth=16, subsample=0.3)
        X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
        model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)
        scores[i] = scores[i] + model.score(X_test, y_test)
        
        booster = model.get_booster()
        tree_df = booster.trees_to_dataframe()
#         print("rysuje drzewo")
#         xgboost.plotting.plot_tree(booster)
        depths = [tree_depth(x) for x in booster.get_dump(dump_format = "json")]
        full_depths[i] = depths
        mean_depths[i] = mean_depths[i] + np.mean(depths)
        stddev_depths[i] = stddev_depths[i] + np.std(depths)
        
    scores[i] = scores[i] / mean
    mean_depths[i] = mean_depths[i] / mean
    stddev_depths[i] = stddev_depths[i] / mean

In [ ]:
print(scores)

In [ ]:
# narysujmy sobie zaleznosc wytrenowania i glebokosci w XGBoost
sns.set()
n = len(scores)
to_plot_scores = [0] * n
to_plot_mean = [0] * n
to_plot_stddev = [0] * n

for i in scores.keys():
    to_plot_scores[i-1] = scores[i]
    to_plot_mean[i-1] = mean_depths[i]
    to_plot_stddev[i-1] = stddev_depths[i]

plt.title("Average score of models")
plt.ylabel("Accuracy in %")
plt.xlabel("Max tree depth")
plt.plot(to_plot_scores)
plt.show()
    
# ax = sns.heatmap(quality)
# plt.show()
# print(to_plot_mean)
# print(to_plot_stddev)

plt.title("Mean depth of tree in ensemble")
plt.ylabel("Mean depth")
plt.xlabel("Max tree depth")
plt.errorbar(range(len(to_plot_mean)), to_plot_mean, to_plot_stddev, linestyle='None', marker='^')
plt.show()

In [ ]:
print(X.values)

In [ ]:
res_shap_cext = {}
res_banz_cext = {}

time_shap_cext = {}
time_banz_cext = {}

model_quality = {}

X,y = shap.datasets.boston()
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

for j in range (2,26,2):
    for i in range(100, 1000, 100):
        model = sklearn.ensemble.RandomForestRegressor(n_estimators=i, max_depth=j)
        model.fit(X_train, y_train)
        
        model_quality[(i, j)] = c_statistic_harrell(model.predict(X_test), y_test)
        
        print("------------------------------")
        print(j, i)
        print("---")
        
        start = time.time()
        res_shap_cext[(i, j)] = TreeCext(model).shap_values(X_test, banz=False)
        time_shap_cext[(i, j)] = time.time() - start
        print(time_shap_cext[(i, j)])

        start = time.time()
        res_banz_cext[(i, j)] = TreeCext(model).shap_values(X_test, banz=True)
        time_banz_cext[(i, j)] = time.time() - start
        print(time_banz_cext[(i, j)])

In [ ]:
time_shap_cext_prop = [ [time_shap_cext[(i, j)] for j in range (2,26,2)] for i in range(100, 1000, 100)]
time_banz_cext_prop = [ [time_banz_cext[(i, j)] for j in range (2,26,2)] for i in range(100
                                                                                        , 1000, 100)]

In [ ]:
time_diffs_banz_shap = \
    [ [ (time_shap_cext[(i, j)] - time_banz_cext[(i, j)]) / time_shap_cext[(i, j)] * 100 \
       for j in range (2,26,2)] for i in range(100, 1000, 100) ]

In [ ]:
plt.figure(figsize=(12, 9))
sns.heatmap(time_diffs_banz_shap, annot=True, fmt=".1f", linewidth=.5,\
           xticklabels=range(2,26,2), yticklabels=range(100, 1000, 100))
plt.title("Percentage of time improvement for TreeShap vs BANZ")
plt.ylabel("No. of trees")
plt.xlabel("Max tree depth")
plt.show()

In [ ]:
plot_qualities = [[model_quality[(i, j)] for j in range (2,26,2)] for i in range(100, 1000, 100)]

plt.figure(figsize=(12, 9))
sns.heatmap(plot_qualities, annot=True, fmt=".3f", linewidth=.5,\
           xticklabels=range(2,26,2), yticklabels=range(100, 1000, 100))
plt.title("C-index quality for model")
plt.ylabel("No. of trees")
plt.xlabel("Max tree depth")
plt.show()

In [ ]:
res_shap_cext_prop = [[res_shap_cext[(i, j)] for j in range (2,26,2)] for i in range(100, 1000, 100)]
res_banz_cext_prop = [[res_banz_cext[(i, j)] for j in range (2,26,2)] for i in range(100, 1000, 100)]
time_shap_cext_prop = [[time_shap_cext[(i, j)] for j in range (2,26,2)] for i in range(100, 1000, 100)]
time_banz_cext_prop = [[time_banz_cext[(i, j)] for j in range (2,26,2)] for i in range(100, 1000, 100)]

In [ ]:
X,y = shap.datasets.nhanesi()
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=7, test_size=0.2)

xgb_full = xgboost.DMatrix(X, label=y)
xgb_train = xgboost.DMatrix(X_train, label=y_train)
xgb_test = xgboost.DMatrix(X_test, label=y_test)

In [ ]:
params = {
    "eta": 0.2,
    "max_depth": 4, 
    "objective": "survival:cox",
    "subsample": 0.5,
}
model_full = xgboost.train(params, xgb_full, 5000, evals = [(xgb_full, "test")], verbose_eval=1000,\
                           early_stopping_rounds=250)

In [ ]:
c_statistic_harrell(model_full.predict(xgb_full), y)

In [ ]:
run_diff_explainers(model_full, xgb_train, xgb_full)

In [ ]:
model = xgboost.XGBRegressor(n_estimators=1000, max_depth=5, subsample=0.3)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

booster = model.get_booster()
tree_df = booster.trees_to_dataframe()

In [ ]:
tree_df

In [ ]:
plt.plot([x * 10 for x in scores.values()])
plt.plot(mean_depths.values())
plt.plot(stddev_depths.values())

plt.show()

# wychodza glebokosci po 13 jak da mu sie wolna reke

In [ ]:
def show_results(attributions):
    a = attributions[1000]
    # print(a[0][1])


#     plt.stackplot(range(len(a[0][1].values[0])), a[0][1].values[0], a[1][1].values[0], a[2][1].values[0],
#                  labels=["Tree", "TreeBanz", "NEW TreeBanz"])
    
    plt.plot(range(len(a[0][1].values[0])), a[0][1].values[0])
    plt.plot(range(len(a[0][1].values[0])), a[1][1].values[0])
    plt.plot(range(len(a[0][1].values[0])), a[2][1].values[0])
    
    plt.legend()
    # plt.subplot(a)
    plt.show()

    shap.plots.bar(a[0][1][0]) #tree
    shap.plots.bar(a[1][1][0]) #treeBanz
    shap.plots.bar(a[2][1][0]) #NEWtreeBanz

In [ ]:
X,y = shap.datasets.boston()
X = X.values

model = xgboost.XGBRegressor(n_estimators=1000, max_depth=32, subsample=0.3)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)
X_eval = X_test[:1]

attributions = run_diff_explainers(model, X_train, X_eval)

# show_results(attributions)
c_statistic_harrell(model, X_test, y_test)

In [ ]:
X,y = shap.datasets.nhanesi()
X = X.values


for i in range(1,15):
    print("i={}".format(i))
    model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000, max_depth=i)
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    model.fit(X_train, y_train)
    X_eval = X_test[:1]

    attributions = run_diff_explainers(model, X_train, X_eval)

    y_predict = model.predict(X_test)
    # show_results(attributions)
    print("c-index:{}".format(c_statistic_harrell(y_predict, y_test)))
    print("mse:{}".format(np.sqrt(MSE(y_predict, y_test))))
    print("----------------------\n")

In [ ]:
x_ax = range(len(y_test))
plt.figure(figsize=(16, 8), dpi=100)
plt.plot(x_ax, y_test, label="original")
plt.plot(x_ax, y_predict, label="predicted")
plt.title("Boston dataset test and predicted data")
plt.xlabel('X')
plt.ylabel('Price')
plt.legend(loc='best',fancybox=True, shadow=True)
plt.grid(True)
plt.show()

In [ ]:
v = {}
for i in range(3):
    v[i] = [(x / attributions[1000][i][1].values[0][0]) for x in attributions[1000][i][1].values[0]]

v

In [ ]:
shap.plots.beeswarm(a[0][1])
shap.plots.beeswarm(a[1][1])

In [ ]:
# # dry run to get all the code warmed up for valid runtime measurements
# for name, exp in explainers:
#     exp(X_eval[:1])

# explain with all the explainers
attributions = [run_explain(name, exp, X_eval) for name, exp in explainers]

In [ ]:
X, y = make_regression(n_samples=400, n_features=16, n_informative=8,random_state=0, shuffle=False)

model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000, max_depth=10)
model.fit(X, y)

In [ ]:
# bst = xgboost.train({"learning_rate": 0.01, "max_depth": 4}, xgboost.DMatrix(X, label=y), 1000)

In [ ]:
# start = time.time()
# shap_values = bst.predict(xgboost.DMatrix(X), pred_contribs=True)
# print(time.time() - start)


In [ ]:

# x = X.head()
# x = X[:1]
x = X
x

In [ ]:
start = time.time()
res_shap_cext = TreeCext(model).shap_values(X, banz=False)
time_shap_cext = time.time() - start
time_shap_cext

In [ ]:
start = time.time()
res_banz_cext = TreeCext(model).shap_values(X, banz=True)
time_banz_cext = time.time() - start
time_banz_cext

In [ ]:
res_shap_cext

In [ ]:
res_banz_cext

In [ ]:
ex = TreeExplainer(model)
start = time.time()
res_shap_py = ex.shap_values(x)
time_shap_py = time.time() - start

In [ ]:
start = time.time()
res_banz_py = ex.banz_values(x)
time_banz_py = time.time() - start

In [ ]:
line = np.asarray(x[:1])
ex.brute_banz(line[0], 0)

In [ ]:
ex.brute_shap(line[0], 0)

In [ ]:
results_dict_cext = {}
results_dict_cext['banz'] = {}
results_dict_cext['shap'] = {}

In [ ]:
for i in range(1, 10):
    X, y = make_regression(n_samples=200 * i, n_features=16, n_informative=8,random_state=0, shuffle=False)

    model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000 + 100 * i, max_depth=10 + i)
    model.fit(X, y)
    
    start = time.time()
    res_shap_cext = TreeCext(model).shap_values(x, banz=False)
    res_shap = time.time() - start
    
    start = time.time()
    res_banz_cext = TreeCext(model).shap_values(x, banz=True)
    res_banz = time.time() - start
    
    print(res_shap)
    print(res_banz)
    results_dict_cext['banz'][i] = res_banz
    results_dict_cext['shap'][i] = res_shap
    

In [ ]:
x = [i for i in results_dict_cext['banz']]
a1 = [results_dict_cext['banz'][i] for i in results_dict_cext['banz']]
a2 = [results_dict_cext['shap'][i] for i in results_dict_cext['shap']]

y_values = {"banz": a1, "treeshap": a2}
labels = ["BANZ", "TREESHAP"]

fig, ax = plt.subplots()
ax.stackplot(x, y_values.values(), labels=labels)

ax.legend(loc='upper left')
plt.show()

In [ ]:
results_dict_py = {}
results_dict_py['banz'] = {}
results_dict_py['shap'] = {}

In [ ]:
for i in range(1, 10):
    X, y = make_regression(n_samples=100 + 25 * i, n_features=6, n_informative=2,random_state=0, shuffle=False)

    model = sklearn.ensemble.RandomForestRegressor(n_estimators=100 + 25 * i, max_depth=4)
    model.fit(X, y)
    
    ex = TreeExplainer(model)
    print('created model')
    start = time.time()
    res_shap_py = ex.shap_values(X)
    time_shap_py = time.time() - start
    
    start = time.time()
    res_banz_py = ex.banz_values(X)
    time_banz_py = time.time() - start
    
    print(res_shap_py)
    print(res_banz_py)
    results_dict_py['banz'][i] = time_banz_py
    results_dict_py['shap'][i] = time_shap_py
    

In [ ]:
x = [i for i in results_dict_py['banz']]
a1 = [results_dict_py['banz'][i] for i in results_dict_py['banz']]
a2 = [results_dict_py['shap'][i] for i in results_dict_py['shap']]

y_values = {"banz": a1, "treeshap": a2}
labels = ["BANZ", "TREESHAP"]

fig, ax = plt.subplots()
ax.stackplot(x, y_values.values(), labels=labels)

ax.legend(loc='upper left')
plt.show()

In [ ]:
len(a1)

In [ ]:
len(a2)